# PEARC 2024 Hands-on Exercises

In this notebook, you will use Tapis v3 to create two systems and one application that will be used to run
a sentiment analysis job on both a VM using docker and an HPC type host using singularity.

To execute each `In[#]` cell, you can click inside the cell and press `Shift + Enter`

Install Tapis Python SDK.  After running the code below you need to restart the runtime - go to the Menu and select Runtime -> Restart runtime or use CTRL+M on the keyboard. Now you can execute the code in the notebook and follow the rest of the tutorial.

In [ ]:
#!pip install tapipy

## Enter training account information

To get things started, please run the following and enter the training account information provided to you:

In [ ]:
import getpass

tenant = 'training'
base_url = 'https://' + tenant + '.tapis.io'

# Enter Tapis Username. Example: pearc24-trainingXX
username = input('Username: ')
#username = 'pearc24-training50'
password = username

## Authenticate and initialize Tapis v3 client

Using this information, you can now use `tapipy` to authenticate in the tenant and initialize the
Tapis v3 client. You should see your token information displayed. This may take a while to run but should take
no more than 30 seconds.

In [ ]:
from tapipy.tapis import Tapis
#Create python Tapis client for user
client = Tapis(base_url= base_url, username=username, password=password)
# *** Tapis v3: Call to Tokens API
client.get_tokens()
# Print Tapis v3 token
client.access_token

## Systems

In this section we create two Tapis systems, one for running on a VM host using FORK and one for running on an HPC type host using BATCH.

Note that although it is possible, we have not provided any login credentials in the system definitions.
Well-crafted system definitions are likely to be copied and re-used, so, for security reasons, it is recommended that
login credentials be registered using separate API calls as discussed below.

### Create a system for the VM host

In [ ]:
# Enter VM password shared with you on email
password_vm = getpass.getpass(prompt='Password for VM: ', stream=None)
# IP address of VM
host = input('Host: ')

In [ ]:
user_id = username
system_id_vm = "pearc24-vm-" + user_id
print(system_id_vm)

# Create the system definition
exec_system_vm = {
  "id": system_id_vm,
  "description": "Test system",
  "systemType": "LINUX",
  "host": host,
  "effectiveUserId":"${apiUserId}",
  "defaultAuthnMethod": "PASSWORD",
  "rootDir": "/",
  "canExec": True,
  "jobRuntimes": [ { "runtimeType": "DOCKER" } ],
  "jobWorkingDir": "HOST_EVAL($HOME)/sharetest/workdir"
}

# Use the client to create the system in Tapis
print("****************************************************")
print("Create system: " + system_id_vm)
print("****************************************************")
client.systems.createSystem(**exec_system_vm)


In [ ]:
# You can also update just a few attributes using the patchSystem call.
# Note that not all attributes may be updated and some attributes, such as *enabled*,
#   may only be updated using a specific call.
# For example, to update the description, first define the json to be used:
patch_system_vm = {
  "description": "System for running jobs on a VM for PEARC24"
}

# Then use the client to make the update:
client.systems.patchSystem(**patch_system_vm, systemId=system_id_vm)

In [ ]:
# List all systems available to you
print("****************************************************")
print("List all systems")
print("****************************************************")
client.systems.getSystems()

In [ ]:
# Get details for the system you created
print("****************************************************")
print("Fetch system: " + system_id_vm)
print("****************************************************")
client.systems.getSystem(systemId=system_id_vm)

### Register Credentials for the VM system

After creating the system, you will need to register credentials for your username. These will be used by Tapis to
access the host. Various authentication methods can be used to access a system, such as PASSWORD and PKI_KEYS. For the
VM a password is used.

In [ ]:
# Register credentials
client.systems.createUserCredential(systemId=system_id_vm, userName=user_id, password=password_vm)

Now you can use the client to list files on the system. This will confirm that the credentials are valid.

In [ ]:
client.files.listFiles(systemId=system_id_vm,path='/')


### Natural Language Processsing: Sentiment Analysis
- Sentiment Analysis is one of the most popular applications of Natural Language Processing, which uses the Text Classification method to analyse the sentiment or emotion of the given text.
- Sentiment analysis assigns a label like 🙂 positive, 🙁 negative, or 😐 neutral to a sequence of text.
- It is useful tool to make business decisions based on customer feedback and reviews.



In [ ]:
#!pip install -q transformers

In [ ]:
from transformers import pipeline

In [ ]:
sentiment_pipeline = pipeline("sentiment-analysis")

In [ ]:
text= "Glad to see you at PEARC"
sentiment_pipeline(text)

### Let's try this with Tapis now

In [ ]:
app_id = "pearc24-sentiment-analysis-" + username
app_def= {
    "id": app_id,
    "version": "0.1",
    "description": "Application utilizing the sentiment analysis model from Hugging Face.",
    "jobType": "FORK",
    "runtime": "DOCKER",
    "containerImage": "tapis/sentiment-analysis:1.0.1",
    "jobAttributes": {
        "parameterSet": {
            "archiveFilter": {
                "includeLaunchFiles": False
            }
        },
        "memoryMB": 1,
        "nodeCount": 1,
        "coresPerNode": 1,
        "maxMinutes": 10
    }
}

In [ ]:
client.apps.createAppVersion(**app_def)

#To update the app
#client.apps.patchApp(appId=app_id, appVersion='0.2', **app_def)

In [ ]:
client.apps.getAppLatestVersion(appId=app_id)

In [ ]:
#Submit job to run the sentiment analysis application
pa= {
    "parameterSet": {
    "appArgs": [
            {"arg": "--sentences"},
            {"arg": "\"This is great\" \"This is not fun\""}
            
        ]
    }}

# Submit a job
job_response_vm=client.jobs.submitJob(name='sentiment analysis',description='sentiment analysis with hugging face transformer pipelines',appId=app_id,appVersion='0.1',execSystemId=system_id_vm, **pa)


In [ ]:
# Get Job submission response
print("****************************************************")
print("Job Submitted: " + app_id)
print("****************************************************")
print(job_response_vm)

In [ ]:
# Get job uuid from the job submission response
print("****************************************************")
job_uuid_vm=job_response_vm.uuid
print("Job UUID: " + job_uuid_vm)
print("****************************************************")

In [ ]:
client.jobs.getJob(jobUuid=job_uuid_vm)

In [ ]:
client.jobs.getJobOutputDownload(jobUuid=job_uuid_vm, outputPath='results.csv')

### Create a system for the HPC cluster

With just a few changes to the system definition you can create a second system that can be used to run the
same application on an HPC type host. Note the minimal changes:

* **id** - A unique id is required
* **host** - Main hostname for the HPC system.
* **rootDir** - Using the root directory of the host gives us flexibility in setting **jobWorkingDir**.
  Note that you still need LINUX permissions.
* **jobWorkingDir** - Now determined dynamically using the Tapis v3 function HOST_EVAL()
* **jobRuntimes** - Most HPC systems support singularity and not docker
* **batchLogicalQueue.hpcQueueName** - HPC queue to use by default.
* **batchLogicalQueues** - HPC queue definitions for this HPC system.

In [ ]:
user_id = username
system_id_hpc = "pearc24-hpc-" + user_id

# Create the system definition
exec_system_hpc = {
  "id": system_id_hpc,
  "description": "System for testing jobs on an HPC type host for PEARC",
  "systemType": "LINUX",
  "host": host,
  "defaultAuthnMethod": "PASSWORD",
  "effectiveUserId": "${apiUserId}",
  "rootDir": "/",
  "canExec": True,
  "jobRuntimes": [ { "runtimeType": "SINGULARITY" } ],
  "jobWorkingDir": "HOST_EVAL($HOME)/sharetest/workdir",
  "canRunBatch": True,
  "batchScheduler": "SLURM",
  "batchSchedulerProfile": "tacc",
  "batchDefaultLogicalQueue": "tapisNormal",
  "batchLogicalQueues": [
    {
      "name": "tapisNormal",
      "hpcQueueName": "normal",
      "maxJobs": 50,
      "maxJobsPerUser": 10,
      "minNodeCount": 1,
      "maxNodeCount": 16,
      "minCoresPerNode": 1,
      "maxCoresPerNode": 68,
      "minMemoryMB": 1,
      "maxMemoryMB": 16384,
      "minMinutes": 1,
      "maxMinutes": 60
    }
  ]
}

# Use the client to create the system in Tapis
print("****************************************************")
print("Create system: " + system_id_hpc)
print("****************************************************")
client.systems.createSystem(**exec_system_hpc)

# If you need to update the system,
# - modify the above definition as needed
# - comment out the above line
# - uncomment the below line
# - re-run the cell
#client.systems.patchSystem(**exec_system_hpc, systemId=system_id_hpc)


In [ ]:
# List all systems available to you
print("****************************************************")
print("List all systems")
print("****************************************************")
client.systems.getSystems()

In [ ]:
# Get details for the system you created
print("****************************************************")
print("Fetch system: " + system_id_hpc)
print("****************************************************")
client.systems.getSystem(systemId=system_id_hpc)

### Register Credentials for the HPC system

As before, now you will need to register credentials for your username. These will be used by Tapis to
access the host.

In [ ]:
password_hpc = password_vm
# Register credentials
client.systems.createUserCredential(systemId=system_id_hpc, userName=username, password=password_hpc)

Now you can use the client to list files on the system. This will confirm that the credentials are valid.

In [ ]:
# List files at the rootDir for the system
path_to_list = "/"
client.files.listFiles(systemId=system_id_hpc, path=path_to_list)

## Application

In order to run a job on a system you will need to create a Tapis application.

### Create an application that can be run on the VM host or the HPC cluster

In [ ]:
app_id_hpc = "pearc24-sentiment-analysis-hpc-" + username
app_def_hpc= {
    "id": app_id_hpc,
    "version": "0.1",
    "description": "Application utilizing the sentiment analysis model from Hugging Face.",
    "jobType": "BATCH",
    "runtime": "SINGULARITY",
    "runtimeOptions": ["SINGULARITY_RUN"],
    "containerImage": "/tmp/sentiment-analysis_1.0.1.sif",
    "jobAttributes": {
            "parameterSet": {
            "archiveFilter": {
                "includeLaunchFiles": False
            }
        },
        "memoryMB": 1,
        "nodeCount": 1,
        "coresPerNode": 1,
        "maxMinutes": 10
    }
}

In [ ]:
client.apps.createAppVersion(**app_def_hpc)

#client.apps.patchApp(**app_def_hpc, appId=app_id_hpc, appVersion='0.1')

In [ ]:
# List all applications available to you
print("****************************************************")
print("List all applications")
print("****************************************************")
client.apps.getApps()

In [ ]:
# Get details for the application you created
print("****************************************************")
print("Fetch application: " + 'Sentiment Analysis HPC app')
print("****************************************************")
client.apps.getAppLatestVersion(appId=app_id_hpc)

In [ ]:
# Submit job to run the sentiment analysis application
pa= {
    "parameterSet": {
    "appArgs": [
            {"arg": "--sentences"},
            {"arg": "\"This is great\" \"This is not fun\""}
            
        ]
    }}

# Submit a job
job_response_hpc=client.jobs.submitJob(name='sentiment analysis',description='sentiment analysis with hugging face transformer pipelines',appId=app_id_hpc,appVersion='0.1',execSystemId=system_id_hpc, **pa)


### Get Job submission response


In [ ]:
# Get Job submission response
print("****************************************************")
print("Job Submitted: " + app_id_hpc)
print("****************************************************")
print(job_response_hpc)

### Get Job UUID from the submission response


In [ ]:
# Get job uuid from the job submission response
print("****************************************************")
job_uuid_hpc=job_response_hpc.uuid
print("Job UUID: " + job_uuid_hpc)
print("****************************************************")

### Check the status of the job


In [ ]:
# Check the status of the job
print("****************************************************")
print(client.jobs.getJobStatus(jobUuid=job_uuid_hpc))
print("****************************************************")

### Download output of the job


In [ ]:
# Once the job is in the FINISHED state, you can download output of the job
print("Job Output file:")

print("****************************************************")
jobs_output_hpc= client.jobs.getJobOutputDownload(jobUuid=job_uuid_hpc,outputPath='results.csv')
print(jobs_output_hpc)
print("****************************************************")

### Setting Notifications on Job events


Note: Make sure to add your email address in the submitJob call.

In [ ]:
pa= {
    "parameterSet": {
    "appArgs": [
            {"arg": "--sentences"},
            {"arg": "\"This is great\" \"This is not fun\""}
            
        ]
    }}

# Submit a job
job_response_hpc_email=client.jobs.submitJob(name='sentiment analysis',description='sentiment analysis with hugging face transformer pipelines',appId=app_id_hpc,appVersion='0.1',execSystemId=system_id_hpc,subscriptions= [ { "description": "Test subscriptions", "eventCategoryFilter": "ALL","deliveryTargets": [ { "deliveryMethod": "EMAIL","deliveryAddress":"<Enter your email>"}] }],**pa)


In [ ]:
# Get Job submission response
print("****************************************************")
print("Job Submitted: " + app_id_hpc)
print("****************************************************")
print(job_response_hpc_email)

In [ ]:
# Get job uuid from the job submission response
print("****************************************************")
job_uuid_hpc_email=job_response_hpc_email.uuid
print("Job UUID: " + job_uuid_hpc_email)
print("****************************************************")

In [ ]:
# Check the status of the job
print("****************************************************")
print(client.jobs.getJobStatus(jobUuid=job_uuid_hpc_email))
print("****************************************************")

### Cancel a job


In [ ]:
# If necessary, you can cancel a long running job.
# To cancel a running job
# client.jobs.cancelJob(jobUuid=job_uuid_vm)

## Share System and App

In [ ]:
#Making your execution system Public
client.systems.shareSystemPublic(systemId=system_id_hpc)

In [ ]:
# Making the app public
client.apps.shareAppPublic(appId=app_id_hpc)

In [ ]:
# Get Share info on the app
client.apps.getShareInfo(appId=app_id_hpc)
# Now any user in the tenant should be able to run your application

In [ ]:
# Unsharing public app
#client.apps.unShareAppPublic(appId=app_id_hpc)

In [ ]:
## You should now be able to run any public apps
'''
pa= {
    "parameterSet": {
    "appArgs": [
            {"arg": "--sentences"},
            {"arg": "\"This is great\" \"This is not fun\""}
            
        ]
    }}

# Submit a job
job_response_hpc_email=client.jobs.submitJob(name='sentiment analysis',description='sentiment analysis with hugging face transformer pipelines',appId=app_id_hpc,appVersion='0.1',execSystemId=system_id_hpc,subscriptions= [ { "description": "Test subscriptions", "eventCategoryFilter": "ALL","deliveryTargets": [ { "deliveryMethod": "EMAIL","deliveryAddress":"<Enter your email>"}] }],**pa)
'''